# District catigorization on various demographic indicators
The goal of this project is to creata more accurate benchmarks for school district preformance. When comparing the preformance of school districts often state or neigboring school districts are used as benchmarks; even when socio-dconomic factors can be extreamly diffrent. Some notable factors that I think may cause districts to preform diffrently is teacher compensation,  teacher student ethnic distributions (speficcily ethnic homogeneity), 

as stronger homoginininty would corilate to stronger outcomes using chorpis christi and the RGV as an example where high student teacher ethnic hogoininity shows high student preformane and outcomes when looking at STAAR or college redinesss scores in hispanic communities leading to high student preformance.



In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Steps
1. Create a data set of all the district refrences for n number of years (I want to see if I can get at least 5 years back)
2. I have to clean them all and then actally make one master list of districts that have been around since 2022 to 2025. Partially because I don't want to clean. but also I think it doens't matter because the change will be so tiny.




Notes--
columns I need to keep
- District_Num = DISTRICT = 6 Digit County District Number	
- District_Name = DISTNAME	= District Name	
- County_Num = COUNTY = 3 Digit County Number
- County_Name = CNTYNAME = County Name
- ESC = REGION = 2 Digit ESC Region	
- ESC_Name = REGNNAME = Region Name	
- Charter = DFLCHART = District 2025 Flag = Charter Operator (Y/N)
- AEA = DFLALTED = District Rated Under AEA Procedures Flag
- District_Rating = D_RATING = 2025 District Overall Grade   -------------------- Pull Score too
- DAD_Post = 2025 District Post Secondary Distinction	
- ASVAB_Status = ASVAB Status PEIMS Description Short
- OUTCOME = District 2025 Special Education Determination Status



## Clean and create Master Districts Dataset


NOTE, for all incoming data I will need to remove districts not in my district_final_cut 

and if I go back more than 2022 if one of my districts in the final cut disapears then I need to remove them from the consideration to.

i.e. 
a district is not listed for any number of reasons before 2019 but in my current list of districts from 2022-2025 if i used data back to 2014 then it will skew... actually I just loaded in 2012 and added that to the set cross refrence and it narrowd down the list a little more so im just going with those ones. 


district_master

In [ ]:
# Load in District data
district_2025 = pd.read_csv(r'District Infromation\2025 District Reference.csv')
district_2024 = pd.read_csv(r'District Infromation\2024 District Reference.csv')
district_2023 = pd.read_csv(r'District Infromation\2023 District Reference Information.csv')
district_2022 = pd.read_csv(r'District Infromation\2022 District Reference Information, Accountability Rating and Special Education Determination Status.csv')
district_2013 = pd.read_csv(r'District Infromation/2013 District Reference .dat', dtype=str)

In [ ]:
# View districts raw data

district_2025.head(2)
# district_2024.head(2)
# district_2023.head(1)
# district_2022.head(1)
district_2013.head()


,DISTRICT,CNTYNAME,COUNTY,DFLALTED,DFLCHART,DISTNAME,D_RATING,REGION,SECS
0,001902,ANDERSON,001,N,N,CAYUGA ISD,M,07,
1,001903,ANDERSON,001,N,N,ELKHART ISD,M,07,
2,001904,ANDERSON,001,N,N,FRANKSTON ISD,M,07,D
3,001906,ANDERSON,001,N,N,NECHES ISD,M,07,
4,001907,ANDERSON,001,N,N,PALESTINE ISD,M,07,


In [4]:
district_25_headers = district_2025.iloc[0]
district_25 = district_2025[1:]
district_25.columns = district_25_headers
district_25.insert(loc=0, column="Year", value="2025")

district_25.head(1)


,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,REGNNAME,DFLCHART,DFLALTED,D_RATING,DAD_POST,ASVAB_STATUS,OUTCOME
1,2025,001902,CAYUGA ISD,001,ANDERSON,07,REGION 07: KILGORE,N,N,B,1,,Meets Requirements


In [5]:
district_2024_headers = district_2024.iloc[0]
district_24 = district_2024[1:]
district_24.columns = district_2024_headers
district_24.insert(loc=0, column="Year", value="2024")

district_24.head(1)

,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,REGNNAME,DFLCHART,DFLALTED,D_RATING,DAD_POST,ASVAB_STATUS,OUTCOME
1,2024,001902,CAYUGA ISD,001,ANDERSON,07,REGION 07: KILGORE,N,N,B,1,,Meets Requirements


In [6]:
district_2023['COUNTY'] = district_2023['COUNTY'].str[1:]
district_2023['REGION'] = district_2023['REGION'].str[1:]
district_2023['DISTRICT'] = district_2023['DISTRICT'].str[1:]
district_2023.insert(loc=0, column="Year", value="2023")

district_23 = district_2023
district_23.head()

,Year,COUNTY,REGION,DISTRICT,DISTNAME,CNTYNAME,DFLCHART,DFLALTED,ASVAB_STATUS
0,2023,001,07,001902,CAYUGA ISD,ANDERSON,N,N,
1,2023,001,07,001903,ELKHART ISD,ANDERSON,N,N,
2,2023,001,07,001904,FRANKSTON ISD,ANDERSON,N,N,ASVAB Alternative Test Offered
3,2023,001,07,001906,NECHES ISD,ANDERSON,N,N,
4,2023,001,07,001907,PALESTINE ISD,ANDERSON,N,N,


In [7]:
district_2022['COUNTY'] = district_2022['COUNTY'].str[1:]
district_2022['REGION'] = district_2022['REGION'].str[1:]
district_2022['DISTRICT'] = district_2022['DISTRICT'].str[1:]
district_2022.insert(loc=0, column="Year", value="2022")

district_22 = district_2022
district_22.head()

,Year,COUNTY,REGION,DISTRICT,D_rating,DISTNAME,CNTYNAME,DFLCHART,DFLALTED,DAD_POST,OUTCOME,asvab_status
0,2022,001,07,001902,A,CAYUGA ISD,ANDERSON,N,N,1,Meets Requirements,
1,2022,001,07,001903,A,ELKHART ISD,ANDERSON,N,N,0,Meets Requirements,
2,2022,001,07,001904,A,FRANKSTON ISD,ANDERSON,N,N,0,Meets Requirements,ASVAB Alternative Test Offered
3,2022,001,07,001906,A,NECHES ISD,ANDERSON,N,N,1,Meets Requirements,
4,2022,001,07,001907,B,PALESTINE ISD,ANDERSON,N,N,0,Needs Assistance,


In [ ]:
district_2013.insert(loc=0, column="Year", value="2013")


In [ ]:
d_25 = set(district_25['DISTRICT'])
d_24 = set(district_24['DISTRICT'])
d_23 = set(district_23['DISTRICT'])
d_22 = set(district_22['DISTRICT'])
d_12 = set(district_2013['DISTRICT'])

# Creating a set of districts only in all
District_final_cut = d_25 & d_24 & d_23 & d_22 & d_12
District_final_cut
len(District_final_cut)


1162

In [10]:
# Check district lenths arent changing  
# 2025: Original List Len: 1208 New List Len: 1208
# 2024: Original List Len: 1207 New List Len: 1207
# 2023: Original List Len: 1209 New List Len: 1209
# 2022: Original List Len: 1207 New List Len: 1207

print(f'Original List Len: {len(district_22['DISTRICT'])} New Set Len: {len(d_22)}')


Original List Len: 1207 New Set Len: 1207


In [11]:
def remove_districts(districts):
    filter = districts['DISTRICT'].isin(District_final_cut)
    filtered_district = districts[filter]
    return filtered_district

In [ ]:
district_25_filtered = remove_districts(district_25)
district_24_filtered = remove_districts(district_24)
district_23_filtered = remove_districts(district_23)
district_22_filtered = remove_districts(district_22)
district_13_filtered = remove_districts(district_2013)


In [ ]:
dist_25_col = set(district_25_filtered.columns)
dist_24_col = set(district_24_filtered.columns)
dist_23_col = set(district_23_filtered.columns)
dist_22_col = set(district_22_filtered.columns)
dist_13_col = set(district_2013.columns)

dist_col_in_all = dist_25_col & dist_24_col & dist_23_col & dist_22_col & dist_13_col
dist_col_in_all

{'CNTYNAME',
 'COUNTY',
 'DFLALTED',
 'DFLCHART',
 'DISTNAME',
 'DISTRICT',
 'REGION',
 'Year'}

In [ ]:
# now I need to make a new data fram that is a version of all of these where all the data is the same, and I want to see what wrong with the ones that arent.
# i.e. if a cdn is around an but the district name is diffrent or county etc

dist_df_list = [district_25_filtered, district_24_filtered, district_23_filtered, district_22_filtered, district_13_filtered]

districts_grouped = pd.concat(
    [df.loc[:, df.columns.intersection(dist_col_in_all)] for df in dist_df_list]   
)

districts_grouped

,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,DFLCHART,DFLALTED
1,2025,001902,CAYUGA ISD,001,ANDERSON,07,N,N
2,2025,001903,ELKHART ISD,001,ANDERSON,07,N,N
3,2025,001904,FRANKSTON ISD,001,ANDERSON,07,N,N
4,2025,001906,NECHES ISD,001,ANDERSON,07,N,N
5,2025,001907,PALESTINE ISD,001,ANDERSON,07,N,N
...,...,...,...,...,...,...,...,...
1223,2012,252902,NEWCASTLE ISD,252,YOUNG,09,N,N
1224,2012,252903,OLNEY ISD,252,YOUNG,09,N,N
1225,2012,253901,ZAPATA COUNTY ISD,253,ZAPATA,01,N,N
1226,2012,254901,CRYSTAL CITY ISD,254,ZAVALA,20,N,N


In [15]:
# Check how many unique 'charter_status' values each district has
charter_check = districts_grouped.groupby('DISTRICT')['DFLALTED'].nunique()

aea_check = districts_grouped.groupby('DISTRICT')['DFLALTED'].nunique()

# IDs where the status changed (more than 1 unique value)
# inconsistent_ids = charter_check[charter_check > 1].index  # Nothing
inconsistent_ids = aea_check[aea_check > 1].index
inconsistent_ids

Index(['015807', '015808', '101804', '101837', '101842', '101864', '193801'], dtype='str', name='DISTRICT')

In [16]:
districts_grouped.shape

districts_cleaned = districts_grouped[~districts_grouped['DISTRICT'].isin(inconsistent_ids)]

print(districts_grouped.shape, districts_cleaned.shape)


(5810, 8) (5775, 8)


In [ ]:
# using 2025 as source of truth

district_master = districts_cleaned[districts_cleaned["Year"]=='2025'].reset_index(drop=True)
district_master = district_master.drop(columns='Year')


district_master.head()

,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,DFLCHART,DFLALTED
0,001902,CAYUGA ISD,001,ANDERSON,07,N,N
1,001903,ELKHART ISD,001,ANDERSON,07,N,N
2,001904,FRANKSTON ISD,001,ANDERSON,07,N,N
3,001906,NECHES ISD,001,ANDERSON,07,N,N
4,001907,PALESTINE ISD,001,ANDERSON,07,N,N


## Clean Student & Staff Datasets

Notes: 
- Seprate student & staff data into their own dataframes

Useful links
- https://rptsvr1.tea.texas.gov/perfreport/tapr/2025/masking.html
- 2013 data download headers explained: https://rptsvr1.tea.texas.gov/perfreport/tapr/2013/download/taprref.html



In [66]:
StudentStaff_Info_2013 = pd.read_csv(r"Staff & Student Data\2013 Staff & Student Information.txt")
#StudentStaff_Info_2013.insert(0, 'Year', '2013')

StudentStaff_Info_2014 = pd.read_csv(r"Staff & Student Data\2014 Staff & Student Information.dat")
#StudentStaff_Info_2014.insert(0, 'Year', '2014')

StudentStaff_Info_2015 = pd.read_csv(r"Staff & Student Data\2015 Staff & Student Information.dat")
#StudentStaff_Info_2015.insert(0, 'Year', '2015')

StudentStaff_Info_2016 = pd.read_csv(r"Staff & Student Data\2016 District Staff & Student Information.dat")
#StudentStaff_Info_2016.insert(0, 'Year', '2016')

StudentStaff_Info_2017 = pd.read_csv(r"Staff & Student Data\2017 Staff & Student Information.dat")
#StudentStaff_Info_2017.insert(0, 'Year', '2017')

StudentStaff_Info_2018 = pd.read_csv(r"Staff & Student Data\2018 Staff, Student, and Annual Graduates.dat")
#StudentStaff_Info_2018.insert(0, 'Year', '2018')

StudentStaff_Info_2019 = pd.read_csv(r"Staff & Student Data\2019 Staff, Student, and Annual Graduates.dat")
#StudentStaff_Info_2019.insert(0,'Year','2019')

StudentStaff_Info_2020 = pd.read_csv(r"Staff & Student Data\2020 Staff, Student, and Annual Graduates.dat")
#StudentStaff_Info_2020.insert(0,'Year','2020')

StudentStaff_Info_2021 = pd.read_csv(r"Staff & Student Data\2021 Staff, Student, and Annual Graduates.csv")
#StudentStaff_Info_2021.insert(0,'Year','2021')

StudentStaff_Info_2022 = pd.read_csv(r"Staff & Student Data\2022 Staff, Student, and Annual Graduates.csv")
#StudentStaff_Info_2022.insert(0,'Year','2022')

StudentStaff_Info_2023 = pd.read_csv(r"Staff & Student Data\2023 Staff, Student, and Annual Graduates.csv")
#StudentStaff_Info_2023.insert(0,'Year','2023')

Student_Info_2024 = pd.read_csv(r"Staff & Student Data\2024 District Student Information.csv")
Staff_Info_2024 = pd.read_csv(r"Staff & Student Data\2024 District Staff Information.csv")
Student_Info_2025 = pd.read_csv(r"Staff & Student Data\2025 District Student Information.csv")
Staff_Info_2025 = pd.read_csv(r"Staff & Student Data\2025 District Staff Information.csv")


In [29]:
#StudentStaff_Info_2013.columns
StudentStaff_Info_2014.columns

Index(['DISTRICT', 'DA0GH13N', 'DA0GR13N', 'DA0GM13N', 'DB0GR13N', 'DH0GR13N',
       'DW0GR13N', 'DI0GR13N', 'D30GR13N', 'D40GR13N',
       ...
       'DPST01SA', 'DPST06SA', 'DPST11SA', 'DPST20SA', 'DPSTTOSA', 'DPSUTOSA',
       'DPSSTOSA', 'DPSCTOSA', 'DPFEIERP', 'DPSAINHP'],
      dtype='str', length=250)

In [63]:
StudentStaff_Info_2013.head()

,DISTRICT,DPETGEEC,DPETGPKC,DPETGKNC,DPETG01C,DPETG02C,DPETG03C,DPETG04C,DPETG05C,DPETG06C,DPETG07C,DPETG08C,DPETG09C,DPETG10C,DPETG11C,DPETG12C,DPETALLC,DPETSPEC,DPETBILC,DPETVOCC,DPETGIFC,DPETLEPC,DPETECOC,DPETNEDC,DPETRSKC,DPETINDC,DPETASIC,DPETPCIC,DPETTWOC,DPETBLAC,DPETHISC,DPETWHIC,DAPID13D,DAPID13N,DAPID13R,DPETDISD,DPETDISC,DPETDISP,DA0GH12N,DA0GR12N,DA0GM12N,DB0GR12N,DH0GR12N,DW0GR12N,DI0GR12N,D30GR12N,D40GR12N,D20GR12N,DS0GR12N,DA0GR12R,DB0GR12R,DH0GR12R,DW0GR12R,DI0GR12R,D30GR12R,D40GR12R,D20GR12R,DS0GR12R,DA0GH12R,DA0GM12R,DPERRAKR,DPERRA1R,DPERRA2R,DPERRA3R,DPERRA4R,DPERRA5R,DPERRA6R,DPERRA7R,DPERRA8R,DPERSAKR,DPERSA1R,DPERSA2R,DPERSA3R,DPERSA4R,DPERSA5R,DPERSA6R,DPERSA7R,DPERSA8R,DAUND12C,DAUND12D,DAUND12R,DPEMALLC,DPEMALLT,DPEMALLP,DPETWHIP,DPETBLAP,DPETHISP,DPETINDP,DPETASIP,DPETPCIP,DPETTWOP,DPETECOP,DPETNEDP,DPETRSKP,DPETLEPP,DPETSPEP,DPETVOCP,DPETBILP,DPETGIFP,DPETGEEP,DPETGPKP,DPETGKNP,DPETG01P,DPETG02P,DPETG03P,DPETG04P,DPETG05P,DPETG06P,DPETG07P,DPETG08P,DPETG09P,DPETG10P,DPETG11P,DPETG12P,DPSTTOFC,DPSUTOFC,DPSSTOFC,DPSCTOFC,DPSETOFC,DPSTTOST,DPSUTOST,DPSSTOST,DPSCTOST,DPST00FC,DPST01FC,DPST06FC,DPST11FC,DPST20FC,DPST00ST,DPST01ST,DPST06ST,DPST11ST,DPST20ST,DPSTNOFC,DPSTBAFC,DPSTMSFC,DPSTPHFC,DPSPCOFC,DPSECOFC,DPSPTOFC,DPSXTOFC,DPSATOFC,DPSOTOFC,DPSTINFC,DPSTPIFC,DPSTASFC,DPSTBLFC,DPSTHIFC,DPSTWHFC,DPSTTWFC,DPSTMAFC,DPSTFEFC,DPSXCOFC,DPSTREFC,DPSTVOFC,DPSTBIFC,DPSTCOFC,DPSTGIFC,DPSTSPFC,DPSTOPFC,DPSTURNN,DPSTURND,DPSTURNR,DPSTTENA,DPSTEXPA,DPSAMIFC,DPCTSOCS,DPCTSOCC,DPCTSCIS,DPCTSCIC,DPCTMATS,DPCTMATC,DPCTG06S,DPCTG06C,DPCTG05S,DPCTG05C,DPCTG04S,DPCTG04C,DPCTG03S,DPCTG03C,DPCTG02S,DPCTG02C,DPCTG01S,DPCTG01C,DPCTGMES,DPCTGMEC,DPCTGKGS,DPCTGKGC,DPCTFLAS,DPCTFLAC,DPCTENGS,DPCTENGC,DPCTSOCA,DPCTSCIA,DPCTMATA,DPCTGMEA,DPCTGKGA,DPCTG06A,DPCTG05A,DPCTG04A,DPCTG03A,DPCTG02A,DPCTG01A,DPCTFLAA,DPCTENGA,DPSPTOFP,DPSTTOFP,DPSUTOFP,DPSSTOFP,DPSCTOFP,DPSETOFP,DPSXTOFP,DPSAMIFP,DPSTNOFP,DPSTBAFP,DPSTMSFP,DPSTPHFP,DPST00FP,DPST01FP,DPST06FP,DPST11FP,DPST20FP,DPSTREFP,DPSTVOFP,DPSTBIFP,DPSTCOFP,DPSTGIFP,DPSTSPFP,DPSTOPFP,DPSTBLFP,DPSTWHFP,DPSTHIFP,DPSTPIFP,DPSTINFP,DPSTASFP,DPSTTWFP,DPSTMAFP,DPSTFEFP,DPSTKIDR,DPST00SA,DPST01SA,DPST06SA,DPST11SA,DPST20SA,DPSTTOSA,DPSUTOSA,DPSSTOSA,DPSCTOSA,DPSAINHP,DPFEIERP
0,1902,4,11,40,42,39,36,42,29,43,50,53,51,66,39,50,595,73,5,180,53,5,227,368,162,0,4,0,30,34,45,482,653,0,0.0,664,5,0.8,30,37,7,5,1,31,0,0,0,0,3,100,13.5,2.7,83.8,0,0,0,0,8.1,81.1,18.9,2.9,3.1,5.4,10,0,0,0,0,2.4,0,50,0,0,0,0,0,0,0,0,329,0,93,598,15.6,81.0,5.7,7.6,0.0,0.7,0.0,5.0,38.2,61.8,27.2,0.8,12.3,30.3,0.8,8.9,0.7,1.8,6.7,7.1,6.6,6.1,7.1,4.9,7.2,8.4,8.9,8.6,11.1,6.6,8.4,54.2,7,3,1.6,11.6,2444371.4,371739.2,197454,115098,1,6,11.8,20,15.4,30872,232733,475936,929229.6,775601,0,47.4,6.8,0,14.8,22.8,65.8,28.8,106,2,0,0,0,2,3,49.2,0,12.8,41.4,3,45.4,3.2,0.4,0,0,5.4,0,5.4,54.2,9.8,7.2,15,14,289,22,286,22,293,25,36,2.6,25.6,2.6,36.2,2.6,36,3,39,2,42,2,0,0,40,3,117,13,353,25,13.2,13,11.8,.,13.4,13.6,9.6,13.6,12,19.6,21,9,14.2,62,51.2,6.6,2.8,1.4,10.8,27.2,13.2,0,87.4,12.6,0,1.8,11,22,36.8,28.2,83.8,5.8,0.6,0,0,9.8,0,3.6,90.8,5.6,0,0,0,0,23.8,76.2,11,30875,38785.6,40045.4,46458.6,50580.4,45082,53006.4,65818,76732,77.6,60.4
1,1903,1,105,59,80,82,80,91,105,92,91,101,98,76,96,79,1236,113,16,315,52,16,636,600,344,2,10,2,47,54,112,1009,1360,1,0.1,1415,5,0.4,54,74,20,3,3,68,0,0,0,0,10,100,4.1,4.1,91.9,0,0,0,0,13.5,73,27,6.2,4.2,1.3,0,0,0,7.8,2.2,0,20,0,11.1,0,0,0,0,0,0,5,595,0.8,179,1208,14.8,81.6,4.4,9.1,0.2,0.8,0.2,3.8,51.5,48.5,27.8,1.3,9.1,25.5,1.3,4.2,0.1,8.5,4.8,6.5,6.6,6.5,7.4,8.5,7.4,7.4,8.2,7.9,6.1,7.8,6.4,96.6,7.6,5,3,15.6,3915026.2,347667.8,300835,234805,14.2,9,21,33,19.4,514659.4,274061.2,752868.2,1439600,933837.4,2,81.2,13.4,0,0,0,112.2,28,155.8,0,0,0,0,1,3,91.6,1,15,81.6,0.6,77.2,5.4,0,0.6,2,11.6,0,19,96.4,19.6,9.4,12.6,7,539,42,472,31,529,36,74,5,103,5,89.6,5,78.6,5,70.4,3.4,59.8,3,0,0,50.6,3.4,112,6,636,53,12.8,15.2,14.8,.,14.8,14.8,20.6,18,15.8,20.6,20,18.8,12,72,62,5,3.2,2,10,18,4.4,2,84,14,0,14.6,9.4,21.8,34,20,

In [26]:
df_staff_25 = pd.DataFrame(Staff_Info_2025.columns)

In [27]:
df_staff_25

,0
0,6 Digit County District Number
1,District Name
2,District 2025 Staff: Teacher Total Full Time E...
3,District 2025 Staff: Support Total Full Time E...
4,District 2025 Staff: School Admin Total Full T...
...,...
113,District 2025 Staff: Teacher Incentive Allotme...
114,District 2025 Staff: Teacher Incentive Allotme...
115,District 2025 Staff: Teacher Incentive Allotme...
116,District 2025 Staff: Teacher Incentive Allotme...


### Side quest, clean staff and student keys